In [26]:
import sys
import collections

from laf.fabric import LafFabric
from etcbc.preprocess import prepare
fabric = LafFabric(verbose = '')

  0.00s This is LAF-Fabric 4.5.21
API reference: http://laf-fabric.readthedocs.org/en/latest/texts/API-reference.html
Feature doc: https://shebanq.ancient-data.org/static/docs/featuredoc/texts/welcome.html



In [27]:
API = fabric.load('etcbc4b','--','clause_stats',

                  {'primary': False,
                   'xmlids':{'node':False,'edge':False},
                   'features':('book chapter verse otype code typ txt monads',''),
                   'prepare': prepare
        
                    }
)
exec(fabric.localnames.format(var='fabric'))

  0.00s LOADING API: please wait ... 
  0.01s USING main  DATA COMPILED AT: 2015-11-02T15-08-56
  4.12s LOGFILE=/Users/Cody/laf-fabric-output/etcbc4b/clause_stats/__log__clause_stats.txt
  4.12s INFO: LOADING PREPARED data: please wait ... 
  4.12s prep prep: G.node_sort
  4.25s prep prep: G.node_sort_inv
  5.02s prep prep: L.node_up
  9.23s prep prep: L.node_down
    18s prep prep: V.verses
    18s prep prep: V.books_la
    18s ETCBC reference: http://laf-fabric.readthedocs.org/en/latest/texts/ETCBC-reference.html
    22s INFO: LOADED PREPARED data
    22s INFO: DATA LOADED FROM SOURCE etcbc4b AND ANNOX lexicon FOR TASK clause_stats AT 2016-03-30T00-20-03


# Song Clause Statistics

In [28]:
def get_ref(ref_type,node):
    if ref_type == 'verse':
        verse = L.u('verse',node)
        return F.verse.v(verse)
    elif ref_type == 'chapter':
        chapter = L.u('chapter',node)
        return F.chapter.v(chapter)
    elif ref_type == 'book':
        book = L.u('book',node)
        return F.book.v(book)

In [29]:
def get_percent(total,freq):
    return str((float(freq / total)*100))+'%'

In [30]:
#provides a function to filter out only the mainline clause types
#for this search, mainline is defined as asyndetic, parallel, coordinate with w or )w and direct speech (999)

def code_filter(node):
    
    code = int(F.code.v(node))
    
    #asyndetic

    if 100 <= code <= 167:
        return True
    
    #parallel
    elif 200 <= code <= 201:
        return True
    
    #asyndetic with conj.
    elif 300 <= code <= 367:
        return True
    
    #syndetic
    elif 400 <= code <= 487:
        return True
    
    #first cl in direct speech
    elif code == 999:
        return True

In [31]:
#indexes all nodes in corpus for faster searches

corpus = ["Canticum"]

cur_book = None
nodes = []

for n in NN():
    if cur_book in corpus:
        nodes.append(n)
    
    if F.otype.v(n) == 'book':
        cur_book = F.book.v(n)
        
msg('{} nodes appended'.format(len(nodes)))

  6.87s 6020 nodes appended


In [36]:
raw_clauses = []

for n in nodes:
    otype = F.otype.v(n)
    if otype == 'clause_atom':
        if code_filter(n) == True:
            raw_clauses.append(F.typ.v(L.u('clause',n)))

msg(str(len(raw_clauses))+' appended')

    53s 389 appended


In [54]:
import re

stats_dict = collections.OrderedDict()

for clause in raw_clauses:
    if clause not in stats_dict:
        stats_dict[clause] = 1
    else:
        stats_dict[clause] += 1
        
stats_dict = collections.OrderedDict(sorted(stats_dict.items(),key=lambda t: t[1],reverse = True))

total = 0
total2 = 0

for item in stats_dict:
    
    get_item = re.findall('Y',item,0)
    if len(get_item) != 0:
        total += stats_dict[item]
        
    get_item2 = re.findall('XY',item,0)
    if len(get_item2) != 0:
        print (item,stats_dict[item])
        total2 += stats_dict[item]
        1
print('fronted: {}'.format(total2))      
print('fronted: {}'.format(get_percent(total,total2)))

WXYq 3
XYqt 2
fronted: 5
fronted: 8.771929824561402%


In [ ]:
simplified = {'way' : ["Way0", "WayX"],
        'nmcl' : ["NmCl","AjCl"],
        'ptc' : ["Ptcp"],
        'qtl' : ["WQt0", "WQtX", "WxQ0", "WXQt", "WxQX", "xQt0", "XQtl", "xQtX", "ZQt0", "ZQtX"],
        'yqtl' : ["WxY0", "WXYq", "WxYX", "WYq0", "WYqX", "xYq0", "XYqt", "xYqX", "ZYq0", "ZYqX"],
        'impv' : ["WIm0","WImX", "WxI0", "WXIm", "WxIX", "xIm0", "XImp", "xImX", "ZIm0", "ZImX"],
        }

In [334]:
Song_Clauses = []
for clause in raw_clauses:
    for key in simplified:
        if clause in simplified[key]:
            Song_Clauses.append(key)

In [335]:
clause_count = {}
total = 0

for clause in Song_Clauses:
    total += 1
    if clause not in clause_count:
        clause_count[clause] = 1
    elif clause in clause_count:
        clause_count[clause] += 1

In [336]:
print (total)
for key in clause_count:
    print ('{:4}{:4}   {}'.format(key, clause_count[key],get_percent(total,clause_count[key])))

328
qtl   84   25.609756097560975%
impv  33   10.060975609756099%
way    2   0.6097560975609756%
nmcl 120   36.58536585365854%
ptc   32   9.75609756097561%
yqtl  57   17.378048780487802%


# Ancient Poetry Clause Statistics

In [337]:
#indexes all nodes in corpus for faster searches

corpus = {'Genesis_49':[str(x) for x in range (1,28)],
          'Exodus_15':[str(x) for x in range (1,22)],
          'Numeri_23':[str(x) for x in range (7,11)]+[str(x) for x in range (18,25)],
          'Numeri_24':[str(x) for x in range (3,10)]+[str(x) for x in range (15,25)],
          'Deuteronomium_32':[str(x) for x in range (1,44)],
          'Deuteronomium_33':[str(x) for x in range (2,30)],
          'Judices_5':[str(x) for x in range (2,32)]
         }

cur_book = None
nodes = []

for n in NN():
    chapter = get_ref('chapter',n)
    verse = get_ref('verse',n)
    if ('{}_{}'.format(cur_book,chapter) in corpus) and verse in corpus['{}_{}'.format(cur_book,chapter)]:
        nodes.append(n)
    
    if F.otype.v(n) == 'book':
        cur_book = F.book.v(n)
        
msg('{} nodes appended'.format(len(nodes)))

    19s 9523 nodes appended


In [338]:
def check_narr(n):
    clause = L.u('clause',n)
    if F.txt.v(clause) != 'N':
        return True

In [339]:
raw_clauses = []

for n in nodes:
    otype = F.otype.v(n)
    if otype == 'clause_atom':
        if code_filter(n) == True and check_narr(n) == True:
            raw_clauses.append(F.typ.v(n))

msg(str(len(raw_clauses))+' appended')


    21s 606 appended


In [340]:
poetry_clauses = []
for clause in raw_clauses:
    for key in simplified:
        if clause in simplified[key]:
            poetry_clauses.append(key)

In [341]:
clause_count = {}
total = 0

for clause in poetry_clauses:
    total += 1
    if clause not in clause_count:
        clause_count[clause] = 1
    elif clause in clause_count:
        clause_count[clause] += 1

In [342]:
print (total)
for key in clause_count:
    print ('{:4}{:4}   {}'.format(key, clause_count[key],get_percent(total,clause_count[key])))

519
qtl  132   25.43352601156069%
impv  38   7.321772639691715%
way   34   6.551059730250482%
nmcl 119   22.928709055876688%
ptc   30   5.780346820809249%
yqtl 166   31.984585741811177%


# Psalms Statistics

In [343]:
#indexes all nodes in corpus for faster searches

corpus = ["Psalmi"]

cur_book = None
nodes = []

for n in NN():
    if cur_book in corpus:
        nodes.append(n)
    
    if F.otype.v(n) == 'book':
        cur_book = F.book.v(n)
        
msg('{} nodes appended'.format(len(nodes)))

    26s 99028 nodes appended


In [344]:
raw_clauses = []

for n in nodes:
    otype = F.otype.v(n)
    if otype == 'clause_atom':
        if code_filter(n) == True:
            raw_clauses.append(F.typ.v(n))

msg(str(len(raw_clauses))+' appended')

    28s 6157 appended


In [345]:
Psalm_Clauses = []
for clause in raw_clauses:
    for key in simplified:
        if clause in simplified[key]:
            Psalm_Clauses.append(key)

In [346]:
clause_count = {}
total = 0

for clause in Psalm_Clauses:
    total += 1
    if clause not in clause_count:
        clause_count[clause] = 1
    elif clause in clause_count:
        clause_count[clause] += 1

In [347]:
print (total)
for key in clause_count:
    print ('{:4}{:4}   {}'.format(key, clause_count[key],get_percent(total,clause_count[key])))

5143
qtl 1149   22.341046082053275%
impv 670   13.027415905113745%
way  333   6.474820143884892%
nmcl 800   15.555123468792534%
ptc  331   6.435932335212911%
yqtl1860   36.16566206494264%


# Chapter by Chapter

Is there a chapter in the Hebrew Bible that resembles the Song of Songs clause statistics? 

<p>way  -   0.61%</p>
<p>yqtl  - 17.38%</p>
<p>nmcl -  46.34%</p>
<p>impv -  10.06%</p>
<p>qtl  -  25.61%</p>

In [395]:
cur_book = None
cur_chapter = None
chapter_check = None
book_check = None

raw_clauses = []
stats = {'total':0}
results = collections.OrderedDict([])

def chron(n):
    if n == 605132:
        return True
    
def amos(n):
    if get_ref('book',n) == 'Obadia' and book_check == 'Amos':
        return True
    
for n in NN():
    
    otype = F.otype.v(n)
    
    if otype == 'book':
        cur_book = F.book.v(n)
        if book_check == None:
            book_check = F.book.v(n)
        
    if otype == 'chapter':
        cur_chapter = F.chapter.v(n)
        if chapter_check == None:
            chapter_check = F.chapter.v(n)
    
    if otype == 'clause_atom':
        if code_filter(n) == True:
            
            if get_ref('chapter',n) == chapter_check and get_ref('book',n) == book_check and chron(n) != True:
                raw_clauses.append(F.typ.v(n))
            
            #initiates data drop
            elif get_ref('chapter',n) != chapter_check or get_ref('book',n) != book_check or chron(n) == True or amos(n) == True:
                
                #checks to see if this is end of HB
                if chron(n) == True:
                    raw_clauses.append(F.typ.v(n))
        
                #assembles frequency data for the chapter
                for clause in raw_clauses:
                    for key in simplified:
                        if clause in simplified[key]:
                            if 'total' in stats:
                                stats['total'] += 1
                            else:
                                stats['total'] = 1
                            if key in stats:
                                stats[key] += 1
                            else:
                                stats[key] = 1
        
                #drops data into results
                for key in stats:
                    if '{}:{}'.format(book_check,chapter_check) in results:
                        results['{}:{}'.format(book_check,chapter_check)].append([key,stats[key]])
                    else:
                        results['{}:{}'.format(book_check,chapter_check)] = [[key,stats[key]]]
                
                #resets variables for next chapter
                book_check = cur_book
                chapter_check = cur_chapter
                raw_clauses = []
                stats = {'total':0}
                
                #logs data for current chapter
                raw_clauses.append(F.typ.v(n))
                
msg(str(len(results))+' chapters logged')

27m 22s 929 chapters logged


In [414]:
results['Obadia:1']

[['total', '100.0%', 55],
 ['impv', '1.8181818181818181%', 1],
 ['qtl', '52.72727272727272%', 29],
 ['nmcl', '5.454545454545454%', 3],
 ['ptc', '5.454545454545454%', 3],
 ['yqtl', '34.54545454545455%', 19]]

In [404]:
for chapter in results:
    for typ in results[chapter]:
        if typ[0] == 'total':
            total = typ[1]
    for typ in results[chapter]:
        typ.append(typ[1])
        typ[1] = get_percent(total,typ[1])

In [405]:
def check_stats(typ,percent):
    rng = 5.00
    qtl_norm = 25.61
    yqtl_norm = 17.38
    nmcl_norm = 36.59
    impv_norm = 10.06
    
    if typ == "nmcl":
        if nmcl_norm-rng <= float(percent.split('%')[0])<= nmcl_norm+rng:
            return True
        
    elif typ == 'yqtl':
        if yqtl_norm-rng <= float(percent.split('%')[0]) <= yqtl_norm+rng:
            return True
            
    elif typ == 'way':
        if 0.00 <= float(percent.split('%')[0]) <= 1.00:
            return True
        
    elif typ == 'qtl':
        if qtl_norm-rng <= float(percent.split('%')[0]) <= qtl_norm+rng:
            return True
        
    elif typ == 'impv':
        if impv_norm-rng <= float(percent.split('%')[0]) <= impv_norm+rng:
            return True

In [406]:
song_matches = []

for chapter in results:
    count = 0
    for typ in results[chapter]:
        if check_stats(typ[0],typ[1]) == True:
            count += 1
    if count >= 4:
        song_matches.append(chapter)
msg(str(len(song_matches))+' found')

28m 03s 1 found


In [407]:
song_matches

['Psalmi:76']

In [408]:
for item in results['Psalmi:76']:
    print ('{}  {}  {}'.format(item[0],item[1],item[2]))

nmcl  33.33333333333333%  8
total  100.0%  24
impv  8.333333333333332%  2
way  4.166666666666666%  1
qtl  29.166666666666668%  7
ptc  8.333333333333332%  2
yqtl  16.666666666666664%  4


In [409]:
for item in results['Canticum:8']:
    print ('{}  {}  {}'.format(item[0],item[1],item[2]))

total  100.0%  43
impv  9.30232558139535%  4
qtl  18.6046511627907%  8
nmcl  23.25581395348837%  10
ptc  9.30232558139535%  4
yqtl  39.53488372093023%  17


# Standard Deviation (for the Song) 

In [426]:
import math

In [411]:
#CORRECT CODE ON THIS! STANDARD DEVIATION CALCULATION IS INCORRECT!

clauses = ['nmcl','qtl','yqtl','way','ptc','impv']

nmcl = collections.OrderedDict()
qtl = collections.OrderedDict()
yqtl = collections.OrderedDict()
ptc = collections.OrderedDict()
impv = collections.OrderedDict()
way = collections.OrderedDict()
total = collections.OrderedDict()



for cls in clauses:
    ct = 1
    while ct < 9:
        for clause in results['Canticum:{}'.format(ct)]:
            if clause[0] == cls:
                eval(cls)['{}'.format(ct)] = clause[2]
            if clause[0] == 'total':
                total['{}'.format(ct)] = clause[2]
            if '{}'.format(ct) not in eval(cls):
                 eval(cls)['{}'.format(ct)] = 0
             
        ct+=1

ct = 1

while ct < 9:
    for clause in clauses:
        eval(clause)['{}'.format(ct)] = get_percent(total['{}'.format(ct)], eval(clause)['{}'.format(ct)])
    ct += 1
ct=1

for clause in clauses:
    
    total = 0.0
    
    for key in eval(clause):
        total += float(eval(clause)[key].split('%')[0])
    
    mean = total/8.00
    
    variance = 0.0
    
    for key in eval(clause):
        variance += math.sqrt((mean - float(eval(clause)[key].split('%')[0]))**2)
    
    print (clause)
    print (variance/8.0)
    print ('\n')

nmcl
11.535884070058888


qtl
7.293212075843452


yqtl
10.709320926965457


way
1.4583333333333335


ptc
5.026292670581553


impv
3.641225342773203




# Standard Deviation (for other texts)

In [522]:
#indexes all books with their chapter lengths for the standard deviation loop
books = collections.OrderedDict()
cur_book = None
chapter_ct = 0

for n in NN():
    otype = F.otype.v(n)
    
    if otype == "chapter":
        
        if cur_book != None:
            if F.book.v(L.u('book',n)) == cur_book:
                chapter_ct += 1
            else:
                books[cur_book] = chapter_ct
                chapter_ct = 1
                cur_book = F.book.v(L.u('book',n))
        else:
            cur_book = F.book.v(L.u('book',n))
            chapter_ct += 1

books['Chronica_II'] = 36

msg(str(len(books))+' books logged')


 1h 59m 04s 39 books logged


In [526]:
deviations = collections.OrderedDict()

for book in books:

    clauses = ['nmcl','qtl','yqtl','way','ptc','impv']

    nmcl = collections.OrderedDict()
    qtl = collections.OrderedDict()
    yqtl = collections.OrderedDict()
    ptc = collections.OrderedDict()
    impv = collections.OrderedDict()
    way = collections.OrderedDict()
    total = collections.OrderedDict()

    chapter_length = books[book]
    book_name = book

    for cls in clauses:
        ct = 1
        while ct <= (chapter_length):
            for clause in results['{}:{}'.format(book_name,ct)]:
                if clause[0] == cls:
                    eval(cls)['{}'.format(ct)] = clause[2]
                if clause[0] == 'total':
                    total['{}'.format(ct)] = clause[2]
                if '{}'.format(ct) not in eval(cls):
                    eval(cls)['{}'.format(ct)] = 0
             
            ct+=1

    ct = 1

    while ct <= (chapter_length):
        for clause in clauses:
            eval(clause)['{}'.format(ct)] = get_percent(total['{}'.format(ct)], eval(clause)['{}'.format(ct)])
        ct += 1

    for clause in clauses:
    
        total = 0.0
    
        for key in eval(clause):
            total += float(eval(clause)[key].split('%')[0])
    
        mean = total/float(chapter_length)
        pre_variance = 0.0
    
        for key in eval(clause):
            pre_variance += (float(eval(clause)[key].split('%')[0])-mean)**2
    
        standard_dev = math.sqrt(pre_variance/float(chapter_length))


        if mean != 0:
            coeff_var = (standard_dev/mean)*100
        else:
            coeff_var = "NA"

        if book in deviations:
            deviations[book].append([clause,coeff_var])
            
        else:
            deviations[book] = [[clause,coeff_var]]
            

msg(str(len(deviations))+' stats logged')

 1h 59m 24s 39 stats logged


In [527]:
deviations

OrderedDict([('Genesis',
              [['nmcl', 78.25431824945255],
               ['qtl', 44.71912940868204],
               ['yqtl', 56.05470996624275],
               ['way', 30.7436059303795],
               ['ptc', 85.00270865386365],
               ['impv', 76.5010351472902]]),
             ('Exodus',
              [['nmcl', 94.23789909775006],
               ['qtl', 46.39585608019654],
               ['yqtl', 72.27342152634819],
               ['way', 68.90367859033805],
               ['ptc', 69.30123797918513],
               ['impv', 86.98807872336957]]),
             ('Leviticus',
              [['nmcl', 66.63193304200597],
               ['qtl', 56.86832553251473],
               ['yqtl', 52.141092516034796],
               ['way', 210.2442928855551],
               ['ptc', 153.29073581869415],
               ['impv', 109.8679928130258]]),
             ('Numeri',
              [['nmcl', 101.41102366806724],
               ['qtl', 63.059960348958874],
               ['yqtl'

# Chapters without Wayyiqtol

In [74]:
song_matches = []

for chapter in results:
    way_count = 0
    for typ in results[chapter]:
        if typ[0] == 'way':
            way_count +=1 
            if 0.00 <= float(typ[1].split('%')[0]) <= 1.00:
                song_matches.append(chapter)
                
    if way_count == 0:
        song_matches.append(chapter)


msg(str(len(song_matches))+' found')

26m 51s 221 found


In [75]:
song_matches2 = []

for chapter in song_matches:
    count = 0
    for typ in results[chapter]:
        if check_stats(typ[0],typ[1]) == True:
            count += 1
    if count >= 2:
        song_matches2.append(chapter)
        
msg(str(len(song_matches2))+' found')

26m 53s 10 found


In [76]:
song_matches2

['Exodus:25',
 'Exodus:27',
 'Leviticus:14',
 'Jeremia:48',
 'Psalmi:14',
 'Psalmi:19',
 'Psalmi:53',
 'Psalmi:111',
 'Psalmi:128',
 'Canticum:4']

In [77]:
results['Exodus:27']

[['qtl', '25.0%', 12],
 ['ptc', '2.08%', 1],
 ['nmcl', '58.33%', 28],
 ['yqtl', '14.58%', 7],
 ['total', '100.0%', 48]]